In [2]:
!pip install ortools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.9/24.9 MB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.0/277.0 kB 15.1 MB/s eta 0:00:00
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0


In [3]:
import pandas as pd
from ortools.linear_solver import pywraplp

# Load the data from your CSV
df = pd.read_csv('/content/drive/MyDrive/UNF/Operation/Bus_dataset3pi.csv')

# Use all routes
num_routes_test = 93
routes = df['Route Number'].values[:num_routes_test]
demand = df[['First Shift', 'Second Shift', 'Third Shift', 'Fourth Shift']].values[:num_routes_test]
trip_factors = df[['Trips of first shift', 'Trips of Second shift', 'Trips of Third shift', 'Trips of Fourth shift']].values[:num_routes_test]
distance = df['Distance'].values[:num_routes_test] # (distance was calculated based in the trip time of a route times the average speed - 30km/h)

# Parameters
fleet_size_type1 = 600
fleet_size_type2 = 90
capacity_type1 = 60
capacity_type2 = 90

# Minimum Trips Required per shift, fix parameter
min_trips_per_shift = [7, 12, 8, 3]  # w1, w2, w3, w4

# Cost parameters for each type of bus in one trip (avg)
cost_type1 = 100
cost_type2 = 150

# Define solver
solver = pywraplp.Solver.CreateSolver('CP-SAT')
if not solver:
    raise Exception("Solver not available!")

# Decision variables (Integer values)
x = {} #trips of type-1 bus
y = {} #trips of type-2 bus
for i in range(len(routes)):
    for j in range(4):
        x[i, j] = solver.IntVar(0, solver.infinity(), f'x_{i}_{j}')
        y[i, j] = solver.IntVar(0, solver.infinity(), f'y_{i}_{j}')

# Shift-specific bus usage variables - count of buses per shift
shift_x = {}
shift_y = {}
for j in range(4):
    shift_x[j] = solver.IntVar(0, solver.infinity(), f'shift_x_{j}')
    shift_y[j] = solver.IntVar(0, solver.infinity(), f'shift_y_{j}')

# Objective function: Minimize total cost
objective_function = solver.Sum(x[i, j] * cost_type1 * trip_factors[i, j] + y[i, j] * cost_type2 * trip_factors[i, j] for i in range(len(routes)) for j in range(4))
solver.Minimize(objective_function)

# Constraints
for i in range(len(routes)):
    for j in range(4):
        # Demand satisfaction constraint
        solver.Add(x[i, j] * capacity_type1 * trip_factors[i, j] + y[i, j] * capacity_type2 * trip_factors[i, j] >= demand[i, j])

# Shift-specific bus usage constraints -
for j in range(4):
    solver.Add(shift_x[j] == solver.Sum(x[i, j] for i in range(len(routes))))
    solver.Add(shift_y[j] == solver.Sum(y[i, j] for i in range(len(routes))))

# Shift-specific fleet size limitations - to not use more tyoe of bus than the allowed
for j in range(4):
    solver.Add(shift_x[j] <= fleet_size_type1)
    solver.Add(shift_y[j] <= fleet_size_type2)

# Minimum trips per shift constraint - for limiting the wait time in 30 minutes in each route
for j in range(4):
    solver.Add(solver.Sum(x[i, j] * trip_factors[i, j] + y[i, j] * trip_factors[i, j] for i in range(len(routes))) >= min_trips_per_shift[j])

# Solve the model
status = solver.Solve()

# Extract and display results
if status == pywraplp.Solver.OPTIMAL:
    print("Optimal solution found!")
    schedule = []
    for i in range(len(routes)):
        for j in range(4):
            total_distance = distance[i] * (x[i, j].solution_value() + y[i, j].solution_value()) * trip_factors[i, j]
            total_trips = (x[i, j].solution_value() + y[i, j].solution_value()) * trip_factors[i, j]
            schedule.append([routes[i], j + 1, x[i, j].solution_value(), y[i, j].solution_value(), total_trips, total_distance])
    df_schedule = pd.DataFrame(schedule, columns=['Route', 'Shift', 'Type1_Buses', 'Type2_Buses', 'Total_Trips', 'Total_Distance'])

    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    pd.set_option('display.max_colwidth', None)

    print(df_schedule)

    total_type1_buses = df_schedule['Type1_Buses'].sum()
    total_type2_buses = df_schedule['Type2_Buses'].sum()
    print(f"Total Type-1 buses used: {total_type1_buses}")
    print(f"Total Type-2 buses used: {total_type2_buses}")

    for j in range(4):
        print(f"Shift {j + 1}: Type-1 buses used = {shift_x[j].solution_value()}, Type-2 buses used = {shift_y[j].solution_value()}")

elif status == pywraplp.Solver.ABNORMAL or status == pywraplp.Solver.NOT_SOLVED:
    print("Solver did not find an optimal solution.")
elif status == pywraplp.Solver.INFEASIBLE:
    print("No feasible solution found. Check constraints.")
elif status == pywraplp.Solver.UNBOUNDED:
    print("The problem is unbounded.")
elif status == pywraplp.Solver.MODEL_INVALID:
    print("The model is invalid.")
else:
    print("Other error occurred.")


Optimal solution found!
     Route  Shift  Type1_Buses  Type2_Buses  Total_Trips  Total_Distance
0        1      1          8.0          0.0         24.0           792.0
1        1      2          1.0          1.0         10.0           330.0
2        1      3          1.0          3.0         16.0           528.0
3        1      4          0.0          2.0          2.0            66.0
4        2      1          4.0          0.0         28.0           406.0
5        2      2          0.0          1.0         12.0           174.0
6        2      3          3.0          0.0         24.0           348.0
7        2      4          0.0          1.0          3.0            43.5
8        3      1          9.0          1.0         20.0           860.0
9        3      2          0.0          2.0          8.0           344.0
10       3      3          0.0          4.0         12.0           516.0
11       3      4          0.0          2.0          2.0            86.0
12       4      1          

In [4]:
print("Max Demand:", demand.max())
print("Max possible capacity type 1:", fleet_size_type1 * trip_factors.max() * capacity_type1)
print("Max possible capacity type 2:", fleet_size_type2 * trip_factors.max() * capacity_type2)

Max Demand: 1968
Max possible capacity type 1: 864000
Max possible capacity type 2: 194400
